In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Read the results which are obtained from watch

In [ ]:
df = pd.DataFrame()
count = 50000

while(True):
    cur = pd.read_csv("../../data/global acc/result" + str(count) + ".csv", skipinitialspace=True, header=None)
    df = df.append(cur, ignore_index=True)
    if count == 500000:
        break
    count += 50000
cur = pd.read_csv("../../data/global acc/result_final.csv", skipinitialspace=True, header=None)
df = df.append(cur, ignore_index=True)

In [ ]:
len(df)

In [ ]:
df.columns = ['global_acc1','global_acc2','global_acc3']

## Add the time, tagname and tester_id into the df

In [ ]:
t = pd.DataFrame.from_csv("../../data/linear_acceleration_grav_mag.csv")
t.columns

In [ ]:
b = t[['tester_id', 'TagName', 'Time']].copy()
for col in ['global_acc1','global_acc2','global_acc3']:
    b[col] = 0

b[['global_acc1','global_acc2','global_acc3']] = df[['global_acc1','global_acc2','global_acc3']].values
b['unix_timestamp'] = pd.to_datetime(b['Time']).values.astype("uint64") / 1000000000
b.to_csv("../../data/global_acc_df.csv")

## Calculate distance and velocity

In [2]:
from scipy.integrate import simps

In [3]:
b = pd.DataFrame.from_csv("../../data/global_acc_df.csv")
b.head()

,tester_id,TagName,Time,global_acc1,global_acc2,global_acc3,unix_timestamp
0,0.0,Tag0,2018-01-08 15:49:46.379666666+08:00,-0.058561,-0.130082,-0.520824,1.515398e+09
1,0.0,Tag0,2018-01-08 15:49:46.384666666+08:00,0.021859,-0.064001,-0.489335,1.515398e+09
2,0.0,Tag0,2018-01-08 15:49:46.389666666+08:00,-0.002340,-0.037200,-0.522459,1.515398e+09
3,0.0,Tag0,2018-01-08 15:49:46.395666666+08:00,-0.070036,0.008021,-0.501289,1.515398e+09
4,0.0,Tag0,2018-01-08 15:49:46.400666666+08:00,-0.069951,0.008737,-0.501289,1.515398e+09


In [4]:
def correct_velocity_drift(v_0):
    d = v_0[-1] / len(v_0)
    for i in range(len(v_0)):
        v_0[i] = v_0[i] - d*i
    return v_0

In [5]:
features = pd.DataFrame(columns=['unix_timestamp','TagName','tester_id',
                                 'v_1','v_2','v_3','d_1','d_2','d_3', 'global_acc1',
                                 'global_acc2','global_acc3','v_12_square','acc_12_square','d_12_square'])


b_groups = b.groupby(['TagName','tester_id'])
count = 0
for key in b_groups.groups.keys():
    
    cur_group = b_groups.get_group(key)
    t = cur_group.unix_timestamp.values
    v_1 = []
    d_1 = []
    v_2 = []
    d_2 = []
    v_3 = []
    d_3 = []
    acc1 = cur_group.global_acc1.values
    acc2 = cur_group.global_acc2.values
    acc3 = cur_group.global_acc3.values
    for i in range(len(t)):
        v_1.append(simps(0.8*acc1[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        v_2.append(simps(0.8*acc2[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        v_3.append(simps(0.8*acc3[:i+1], x=t[:i+1]))
        
    v_1 = correct_velocity_drift(v_1)
    v_2 = correct_velocity_drift(v_2)
    v_3 = correct_velocity_drift(v_3)
    for i in range(len(t)):
        d_1.append(simps(v_1[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        d_2.append(simps(v_2[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        d_3.append(simps(v_3[:i+1], x=t[:i+1]))
    
    v_1 = np.array(v_1)
    d_1 = np.array(d_1)
    v_2 = np.array(v_2)
    d_2 = np.array(d_2)
    v_3 = np.array(v_3)
    d_3 = np.array(d_3)

    
    
    temp_df = pd.DataFrame()
    
    temp_df['unix_timestamp'] = t
    temp_df['TagName'] = np.array([key[0]]*len(t))
    temp_df['tester_id'] = np.array([key[1]]*len(t))
                                                           
        
    temp_df['v_1'] = v_1
    temp_df['v_2'] = v_2
    temp_df['v_3'] = v_3
    
    temp_df['d_1'] = d_1                                       
    temp_df['d_2'] = d_2
    temp_df['d_3'] = d_3
    temp_df['global_acc1'] = acc1
    temp_df['global_acc2'] = acc2
    temp_df['global_acc3'] = acc3
    
    temp_df['v_12_square'] = np.sqrt(v_1*v_1 + v_2*v_2)
    temp_df['acc_12_square'] = np.sqrt(acc1*acc1 + acc2*acc2)
    temp_df['d_12_square'] = np.sqrt(d_1*d_1 + d_2*d_2)

    features = features.append(temp_df, ignore_index=True)
    count+=1
    if (count%50==0):
        print(count)

    

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000


In [6]:
features.head()

,unix_timestamp,TagName,tester_id,v_1,v_2,v_3,d_1,d_2,d_3,global_acc1,global_acc2,global_acc3
0,1.515398e+09,Tag0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,-0.058561,-0.130082,-0.520824
1,1.515398e+09,Tag0,0.0,0.000478,-0.000150,-0.000233,0.000001,-3.744747e-07,-5.831835e-07,0.021859,-0.064001,-0.489335
2,1.515398e+09,Tag0,0.0,0.001138,-0.000088,-0.000427,0.000005,-1.144617e-06,-2.266147e-06,-0.002340,-0.037200,-0.522459
3,1.515398e+09,Tag0,0.0,0.001495,0.000063,-0.001139,0.000013,-1.162659e-06,-6.740278e-06,-0.070036,0.008021,-0.501289
4,1.515398e+09,Tag0,0.0,0.001741,0.000383,-0.001300,0.000021,-3.033544e-07,-1.351024e-05,-0.069951,0.008737,-0.501289


In [7]:
features.to_csv("../../data/global_acc_features_df.csv")

In [ ]:
features = pd.DataFrame.from_csv("../../data/global_acc_features_df.csv")

In [8]:
import math


In [9]:
groups = features.groupby(['TagName','tester_id'])
count = 0
features['v_12_square'] = 0
features['acc_12_square'] = 0
features['d_12_square'] = 0
for key in groups.groups.keys():
    cur_group = groups.get_group(key)

    acc1 = cur_group.global_acc1.values
    acc2 = cur_group.global_acc2.values


    features.loc[(features.TagName == key[0]) & (features.tester_id == key[1]),'v_12_square'] = v_12_square
    features.loc[(features.TagName == key[0]) & (features.tester_id == key[1]),'acc_12_square'] = acc_12_square
    features.loc[(features.TagName == key[0]) & (features.tester_id == key[1]),'d_12_square'] = d_12_square

    count+=1
    if (count%50==0):
        print(count)

    

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000


In [10]:
features.to_csv("../../data/global_acc_features_df.csv")